# Avocado Classifier
This Jupyter Notebook contains the code that takes in a table with information about avocados (average price, total volumne, total bags) and classifies them as either `conventional` or `organic`. This classifier is a $k$-nearest neighbors classifier using the cartesian distance between the point in question and the points in the training set. The data set is from Kaggle (https://www.kaggle.com/neuromusic/avocado-prices).

## 1. Import datascience, numpy, and the table
The cell below imports the `datascience` and `numpy` libraries of Python, as well as opens the csv file as a `datascience` Table object.

In [1]:
import pandas as pd
import numpy as np

avocado = pd.read_csv('avocado.csv')
avocado.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


## 2. Divide the Kaggle data set into the training and test sets
This cell selects the 4 columns we will use from the original table (three data point columns and the type column) and shuffles the rows of the csv file and separates them into a training set, to which the avocado to be classified will be compared, and a test set, to test the accuracy of the classifer once it is built. The test set will retain its `type` column so that we know what proportion of avocados the classifier gets correct. The training set has 18,000 rows and the test set has 249.

In [2]:
av = avocado[['AveragePrice', 'Total Volume', 'Total Bags', 'type']]
av = av.sample(n=len(av))
av_train = av.iloc[0:18000,]
av_test = av.iloc[18001:,]
av_train.shape, av_test.shape

((18000, 4), (248, 4))

## 3. Define a function to find the cartesian distances
In this section, I will define a function that finds the 3-dimensional cartesian distant between two points. This is an application of the Pythagorean Theorem. The distance between two points $(x_1, y_1, z_1)$ and $(x_2, y_2, z_2)$ is

$$d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2 + (z_2-z_1)^2}$$

The function defined takes as arguments a table whose first 3 columns are data points and an array containing the corresponding values for the point that is being compared. It returns the table with a new column that has the distance between each row in the table and the point in the array.

In [16]:
def dist(df, arr):
    '''Takes in a table where the 1st 3 columns are the numerical data
    and returns the cartesian distance from an array with coincident values'''
    
    # turn 1st 3 columns of df into arrays
    arr_1 = np.array(df.iloc[:,0])
    arr_2 = np.array(df.iloc[:,1])
    arr_3 = np.array(df.iloc[:,2])
    
    # iterate through 3-tuples of training set data and create array of distances
    dists = []
    for i in zip(arr_1, arr_2, arr_3):
        dist = np.sqrt((arr[0] - i[0])**2 + (arr[1] - i[1])**2 + (arr[2] - i[2])**2)
        dists += [dist]
        
    new_df = df.copy().reset_index(drop=True)
    new_df['distances'] = pd.Series(dists)
        
    return new_df

# demonstrate dist function
dist(av_train, av_test.iloc[0,0:3].values).head()

,AveragePrice,Total Volume,Total Bags,type,distances
0,1.38,3372297.92,1078418.77,conventional,3.534689e+06
1,1.83,7301.30,6664.25,organic,5.476072e+03
2,1.53,2099.77,332.83,organic,3.743650e+03
3,1.52,8221.86,7661.85,organic,6.727099e+03
4,1.16,151456.08,82460.15,conventional,1.667828e+05


## 4. Define a function to find the majority classification
$k$-NN classifiers work by determining what classification a majority of the $k$ points closest to a point in question have. The function `find_majority` defined below runs the `dist` function on a table and returns that output sorted by increasing distance. The function `knn` below that selects the top $k$ rows and returns the majority classification.

In [17]:
def find_majority(df_train, df_test, row_index):
    '''Takes in training table (t), test table (t2), and row index of test
    table value (row_index) and computes the cartesian distance then
    returns the training table sorted by incrasing distance'''
    test = df_test.iloc[row_index,:3]
    d = dist(df_train, test)
    return d.sort_values('distances')

find_majority(av_train, av_test, 0).head()

,AveragePrice,Total Volume,Total Bags,type,distances
12985,1.50,5744.26,1418.78,organic,68.390015
7503,1.59,5737.50,1483.66,organic,77.485473
17565,1.67,5760.76,1438.19,organic,83.166065
10465,1.66,5614.09,1369.36,organic,90.968168
2717,1.79,5581.18,1483.26,organic,108.160252


In [21]:
def knn(df_train, df_test, row_index, k):
    sort = find_majority(df_train, df_test, row_index)
    df = sort.iloc[:k,].groupby('type').count().sort_values('distances', ascending=False)
    return df.index[0]

row_index = 0
classification = knn(av_train, av_test, row_index, 7)
actual = av_test['type'].iloc[0]
print(f"Row {row_index} Classification: {classification}\nRow {row_index} Actual Type: {actual}")

Row 0 Classification: organic
Row 0 Actual Type: organic


## 5. Test the accuracy of the 7-NN classifier
For an example, I will text how accurate the 7-nearest neighbors classifer is. The `test_accuracy` function defined below runs the classifier on all rows of the `av_test` table (the entire test set), and then returns the proportion of rows that were correctly classified.

In [25]:
def test_accuracy(train, test, k):
    '''Returns proportion of correct classifications from avocado classifier'''
    classed = []
    for i in np.arange(test.shape[0]):
        cl = knn(train, test, i, k)
        classed += [cl]
    
    return np.count_nonzero(classed == test['type']) / test.shape[0]

test_accuracy(av_train, av_test, 7)

KeyboardInterrupt: 

## 6. Determining the optimal value of $k$
In order to determine how many nearest neigbors would be best to run on a random avocado, this second determines the optimal value of $k$ based on the training set. It will run through the classifier for odd integer values 1 through 99, and create a DataFrame where the row index is $k$ and the `accuracy` column contains the percentage of correct classifications for that value of $k$.

In [ ]:
results = []
for i in np.arange(1, 100, 2):
    result = test_accuracy(av_train, av_test, i)
    results += [result]
    
optimal_k = pd.DataFrame(
    data = {
        'accuracy' : results
    },
    index = [k for k in np.arange(1, 100, 2)]
)
optimal_k.sort_values('accuracy', ascending=False)

Based on the table above, it seems that using 7, 13, or 49 for $k$ are all equally as accurate (with minor, neglible differences, presumably). 